<a href="https://colab.research.google.com/github/liuying-y/exam1/blob/main/bert_gan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-models-official
!pip install tensorflow_hub
!pip install tensorflow_text

     |████████████████████████████████| 2.2 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 77.7 MB/s 
     |████████████████████████████████| 47.8 MB 1.9 MB/s 
     |████████████████████████████████| 99 kB 9.3 MB/s 
     |████████████████████████████████| 90 kB 8.9 MB/s 
     |████████████████████████████████| 636 kB 73.9 MB/s 
     |████████████████████████████████| 352 kB 79.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 4.9 MB 63.1 MB/s 
     |████████████████████████████████| 1.1 MB 68.7 MB/s 
     |████████████████████████████████| 237 kB 64.6 MB/s 
     |████████████████████████████████| 462 kB 60.0 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=a9f23ba14a5c0ea88c2ff9f41c60d2b7c765ef835dc3069f703c16ce9b68cfb9
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
from numpy import expand_dims,zeros,ones,asarray
from numpy.random import randn,randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape , Flatten, Conv2D,Conv2DTranspose
from keras.layers import LeakyReLU, Dropout, Lambda, Activation 
from matplotlib import pyplot
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend 
from keras.utils.np_utils import *
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

import os 
from google.colab import drive
#drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive')

def read_data ( ):
    # 读取训练集
    dataset = pd.read_csv(r"data_process/split1new_s3.csv")
    benign = pd.read_csv(r"data_process/benign/benign1.csv")
    merge = dataset.append (benign)
    trainX = merge.copy(deep = True)
    #print ( merge.shape )
    print ( trainX.shape )
    dataset = pd.read_csv(r"data_process/2/split2new_s3.csv")
    benign = pd.read_csv(r"data_process/benign/benign2.csv")
    merge = dataset.append (benign)
    trainX = trainX.append (merge )
    #print ( merge.shape )
    print ( trainX.shape )
    
    # 读取测试集
    dataset = pd.read_csv(r"data_process/3/split3new_s3.csv")
    benign = pd.read_csv(r"data_process/benign/benign3.csv")
    testX = dataset.append (benign)
    print ( testX.shape )
   # print ( trainx.shape )
    return trainX,testX

def pre_process ( trainX,testX ):
    
    labelt=trainX.Label.copy()
    arrs=labelt.values
    d1=np.array(arrs)
    values = array(d1)
    #print(values)
    # integer encode
    label_encoder = LabelEncoder()
    trainY = label_encoder.fit_transform(values)
    print (',,')
    print ( trainY.shape)
    
    labelt=testX.Label.copy()
    arrs=labelt.values
    d1=np.array(arrs)
    values = array(d1)
    testY = label_encoder.fit_transform(values)
    print ( testY.shape)
    
    trainx=['']*trainX.shape[0]
    '''
    row2=trainX.iloc[[0],0:77]
    rowp2=np.array(row2)
    tep= ' '.join([str(x) for x in rowp2[0]])
    print (tep)
    '''  
    
    for r in range (0,trainX.shape[0]):
        row2=trainX.iloc[[r],0:77]
        rowp2=np.array(row2)
        trainx[r]= ' '.join([str(x) for x in rowp2[0]])
        
    testx = ['']*testX.shape[0]
    for r in range (0,testX.shape[0]):
        row2=testX.iloc[[r],0:77]
        rowp2=np.array(row2)
        testx[r]= ' '.join([str(x) for x in rowp2[0]])
        
    trainx,testx = asarray(trainx), asarray(testx)
    print ('....')
    print (trainx.shape)
    print ( testx.shape)
    
    trainset = (trainx,trainY)
    testset = (testx,testY)
    return (trainset, testset)


def data_process2 (trainX,testX ):
    labelt=trainX.Label.copy()
    arrs=labelt.values
    d1=np.array(arrs)
    values = array(d1)
    #print(values)
    # integer encode
    label_encoder = LabelEncoder()
    trainY = label_encoder.fit_transform(values)
    print (',,')
    print ( trainY.shape)
    
    labelt=testX.Label.copy()
    arrs=labelt.values
    d1=np.array(arrs)
    values = array(d1)
    testY = label_encoder.fit_transform(values)
    print ( testY.shape)
    
    trainx=['']*trainX.shape[0]
    '''
    row2=trainX.iloc[[0],0:77]
    rowp2=np.array(row2)
    tep= ' '.join([str(x) for x in rowp2[0]])
    print (tep)
    '''  
    
    for r in range (0,trainX.shape[0]):
        row2=trainX.iloc[[r],0:77]
        rowp2=np.array(row2)
        trainx[r]= ' '.join([str(x) for x in rowp2[0]])
        
    testx = ['']*testX.shape[0]
    for r in range (0,testX.shape[0]):
        row2=testX.iloc[[r],0:77]
        rowp2=np.array(row2)
        testx[r]= ' '.join([str(x) for x in rowp2[0]])
        
    trainx,testx = asarray(trainx), asarray(testx)
    print ('....')
    print (trainx.shape)
    print ( testx.shape)
    x1 = pd.DataFrame(trainx) # header:原第一行的索引，index:原第一列的索引
    x1.to_csv(r"data/trainx.csv",header=None,index=0)
    t1 = pd.DataFrame(testx) # header:原第一行的索引，index:原第一列的索引
    t1.to_csv(r"data/testx.csv",header=None,index=0)

    x1_y = pd.DataFrame(trainY) # header:原第一行的索引，index:原第一列的索引
    x1_y.to_csv(r"data/trainY.csv",header=None,index=0)
    t1_y = pd.DataFrame(testY) # header:原第一行的索引，index:原第一列的索引
    t1_y.to_csv(r"data/testY.csv",header=None,index=0)

    trainset = (trainx,trainY)
    testset = (testx,testY)
    return (trainset, testset)
def read_peocessed_data ( ):
    # 读取训练集
    x1 = pd.read_csv(r"data/trainx.csv",header=None)
    t1 = pd.read_csv(r"data/testx.csv",header=None)
    print (x1.shape)
    print ( t1.shape)
    x1_y = pd.read_csv(r"data/trainY.csv",header=None)
    t1_y = pd.read_csv(r"data/testY.csv",header=None)
    print (x1_y.shape)
    print ( t1_y.shape)
    trainX = np.array ( x1)
    testX = np.array ( t1)
    trainY = np.array ( x1_y)
    testY = np.array (t1_y )
    ytest = to_categorical(testY)
    trainset = (trainX, trainY)
    testset = (testX, ytest)

    print (",,,,")
    print ( trainX.shape )
    print ( trainY.shape)
    print ( testX.shape)
    print ( testY.shape)
    return trainset,testset


def select_samples (dataset, n_classes=10 ):
    
    ratio = [50,5,8,5,3,12,3,4,5,5]
    X, y = dataset
    X_list, y_list = list(), list()
  #  n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), ratio[i])
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    
    ytrain = to_categorical(asarray(y_list))
    return asarray(X_list), ytrain
    

class SGAN:
    def __init__(self):
        self.width = 24
        self.length = 32
        self.channels = 1
        self.shape = (self.width, self.length, self.channels)
        self.latent_dim = 100
        self.tfhub_handle_preprocess = 'https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_preprocess/3'
        #'https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
        #'https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_preprocess/3'
        #https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_L-12_H-768_A-12/3
        self.tfhub_handle_encoder = 'https://hub.tensorflow.google.cn/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'
        #'https://hub.tensorflow.google.cn/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
        
        self.d_model = self.build_discriminator ( )
        self.g_model = self.build_generator ( )
        in_put1 = Input(shape=(), dtype=tf.string, name='text')
        
        self.d_model.trainable =  False
        sgan_outlayer = self.d_model ([ in_put1 ,self.g_model.output] )
        #(inputs1 , inputs2) = self.d_model.inputs
        (output1, output2, output3) = self.d_model.outputs
        self.sgan = Model ( inputs =[in_put1,self.g_model.input] ,outputs = sgan_outlayer)
        self.sgan.compile ( loss='binary_crossentropy',optimizer=Adam(lr= 0.0002, beta_1=0.5))
        
        #from keras.utils.vis_utils import plot_model
        #keras.utils.plot_model(model, to_file='graph.png', show_shapes=True)
        
        #tf.keras.utils.plot_model(self.sgan, to_file='sgan.png',show_shapes=True)
        
    def build_discriminator(self ):
    
        in_put = Input(shape=(), dtype=tf.string, name='text')
        
        # bert 特征提取层
        preprocessing_layer = hub.KerasLayer(self.tfhub_handle_preprocess, name='preprocessing')
        encoder_inputs = preprocessing_layer(in_put)
        encoder = hub.KerasLayer(self.tfhub_handle_encoder, trainable=True, name='BERT_encoder')
        outputs = encoder(encoder_inputs)
        net = outputs['pooled_output']
        net = Dropout(0.1)(net)
        
        epochs = 5
        steps_per_epoch = 512
        num_train_steps = steps_per_epoch * epochs
        num_warmup_steps = int(0.1*num_train_steps)

        init_lr = 3e-5
        optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
       
        # 定义 分类器 输出
        c_out = Dense ( 10) ( net )
        c_out = Activation('softmax' )(c_out)
        #c_model = Model ( in_put, c_out)
       # c_model.compile (loss='sparse_categorical_crossentropy',optimizer =optimizer,metrics= ['accuracy'] )
        
        
        # 定义判别器 
        #generator = self.build_generator ( )
        g_input = Input ( shape = (768,))
        layerd = Dense(1)
        gd_out = layerd(g_input)
        gd_out = Activation('sigmoid')(gd_out)
        d_out = layerd ( net)
        d_out = Activation('sigmoid')(d_out)
        '''
        d_model = Model ( in_put, d_out)
        gd_model = Model ( g_input, gd_out)
        gd_model.compile ( loss='binary_crossentropy',optimizer = Adam (lr=0.0002, beta_1 = 0.5), metrics = ['accuracy'])
        d_model.compile ( loss='binary_crossentropy',optimizer = optimizer, metrics = ['accuracy'])
        '''
        #[c_out, d_out,gd_out]
        discriminator = Model(inputs = [in_put, g_input],outputs ={'c_output':c_out, 'd_output':d_out,'gd_output':gd_out} )
        discriminator.compile ( loss ={'c_output':'categorical_crossentropy','d_output':'binary_crossentropy','gd_output':'binary_crossentropy' },
                               optimizer = optimizer,
                               metrics = ['accuracy']
                            )
        # optimizer,optimizer,Adam (lr=0.0002, beta_1 = 0.5)
        #print ( "this is c_model")
        #c_model.summary()
        #print ( "this is d_model")
        discriminator.summary()
        tf.keras.utils.plot_model(discriminator, to_file='discriminator.png',show_shapes=True)
        return discriminator
    
    def build_generator (self ):
        in_put = Input ( shape = (self.latent_dim,) )
        
        gen = Dense ( 128*6*8)(in_put)
        gen = LeakyReLU ( alpha = 0.2 ) ( gen)
        gen = Reshape ((6,8,128))(gen)
        
        gen = Conv2DTranspose ( 128, (4,4),strides=(2,2),padding = 'same')(gen )
        gen = LeakyReLU (alpha = 0.2) ( gen )
        gen = Conv2DTranspose ( 128,(4,4),strides=(2,2),padding = 'same') (gen)
        gen = LeakyReLU (alpha = 0.2) (gen )
        
        gen = Conv2D ( 1, (7,7), activation ='tanh',padding = 'same')(gen)
        gen = Reshape ((768,))(gen)
        model = Model ( in_put, gen)
        print ( "this is g_model")
        model.summary()
        return model
    
    def summarize_performance ( self,step, dataset, n_samples=100):
        # prepare fake examples 
        #X ,_ = generate_fake_samples(g_model,latent_dim,n_samples)
        # scale from [-1,1] to [0,1]
        X , y = dataset
        valid = np.ones((X.shape[0], 1))
        fake = np.zeros((X.shape[0], 1))
        noise = np.random.normal(0, 1, (X.shape[0], self.latent_dim))
        gen_sample = self.g_model.predict(noise)
       
        d_losst,d_loss1,d_loss2,d_loss3,acc,d_acc1,d_acc2 = self.d_model.evaluate([X,gen_sample],{'c_output':y, 'd_output':valid,'gd_output':fake} , verbose=0)
        print ('Classifier Accuracy : %.3f%%' % (acc * 100))
        # save the generator model 
        filename2 = 'g_model_%04d.h5' % (step+1)
        self.g_model.save(filename2)
        # save the calssifier model
        filename3 = 'd_model_%04d.h5' % (step+1)
        self.d_model.save(filename3)
        print ( '>saved :  ,%s  ,and %s '%( filename2, filename3))
        
        
    def train (self,epochs, batch_size , sample_interval,dataset,testset):
        
        X_train , y_train = dataset
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        
        for i in range(epochs):
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_sample = self.g_model.predict(noise)
            
            tx1,ty1 = select_samples( dataset)
         
            
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            tx2 = X_train[idx]
            
            d_losst,d_loss1,d_loss2,d_loss3,c_acc,d_acc1,d_acc2 = self.d_model.train_on_batch ([tx1,gen_sample],
                                                  {'c_output':ty1, 'd_output':valid,'gd_output':fake}            )
          
            d_loss = 0.5 * np.add(d_loss2, d_loss3)
            
            tx2,ty2 = select_samples( dataset)
            noise2 = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.sgan.train_on_batch([tx2,noise2],{'c_output':ty1, 'd_output':valid,'gd_output':fake})
            #[ty2, valid,fake]
            print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, d_loss1, c_acc*100, d_loss2, d_loss3, g_loss[0]))
            
            if (i+1) % (sample_interval * 1) == 0:
                #sample_images(i,g_model,latent_dim,)
                self.summarize_performance(i,testset)
 
            
            
        
#if __name__ == '__main__':
#sgan = SGAN()
#sgan.train(epochs=8401, batch_size=100, sample_interval=400,dataset=dataset)        
'''   
trainX, testX = read_data()
trainSet , testSet  = data_process2 ( trainX, testX)'''
trainSet , testSet = read_peocessed_data()
#sgan.train ( epochs=8000, batch_size=100 , sample_interval=4000,dataset= trainSet)

(715792, 1)
(361586, 1)
(715792, 1)
(361586, 1)
,,,,
(715792, 1)
(715792, 1)
(361586, 1)
(361586, 1)


In [ ]:
sgan = SGAN()
sgan.train ( epochs=2000, batch_size=100 , sample_interval=500,dataset= trainSet,testset = testSet)
#sgan.summarize_performance(499,testSet)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, c[2.316,15], d[0.414,0.697], g[1.916]
>2, c[2.274,18], d[0.405,0.661], g[1.877]
>3, c[2.277,14], d[0.413,0.620], g[1.822]
>4, c[2.309,16], d[0.406,0.583], g[1.794]
>5, c[2.273,18], d[0.392,0.541], g[1.743]
>6, c[2.297,14], d[0.400,0.492], g[1.681]
>7, c[2.314,15], d[0.391,0.441], g[1.654]
>8, c[2.350,9], d[0.375,0.388], g[1.570]
>9, c[2.274,19], d[0.379,0.331], g[1.503]
>10, c[2.272,21], d[0.369,0.279], g[1.448]
>11, c[2.270,19], d[0.348,0.235], g[1.379]
>12, c[2.282,19], d[0.351,0.190], g[1.301]
>13, c[2.201,25], d[0.335,0.157], g[1.285]
>14, c[2.251,19], d[0.300,0.129], g[1.233]
>15, c[2.219,25], d[0.298,0.110], g[1.216]
>16, c[2.274,24], d[0.280,0.094], g[1.189]
>17, c[2.213,25], d[0.271,0.082], g[1.171]
>18, c[2.200,33], d[0.252,0.072], g[1.137]
>19, c[2.149,25], d[0.251,0.066], g[1.118]
>20, c[2.222,29], d[0.260,0.058], g[1.110]
>21, c[2.195,23], d[0.245,0.053], g[1.097]
>22, c[2.150,34], d[0.227,0.048], g[1.074]
>23, c[2.075,43], d[0.231,0.045], g[1.069]
>24, c[2.089,42], d[0

>saved :  ,g_model_0500.h5  ,and d_model_0500.h5 
>501, c[0.021,99], d[0.001,0.000], g[0.471]
>502, c[0.016,100], d[0.001,0.000], g[0.472]
>503, c[0.024,99], d[0.001,0.000], g[0.480]
>504, c[0.068,99], d[0.001,0.000], g[0.479]
>505, c[0.021,99], d[0.001,0.000], g[0.504]
>506, c[0.084,98], d[0.001,0.000], g[0.490]
>507, c[0.069,97], d[0.001,0.000], g[0.493]
>508, c[0.058,98], d[0.001,0.000], g[0.481]
>509, c[0.118,95], d[0.001,0.000], g[0.482]
>510, c[0.041,98], d[0.001,0.000], g[0.458]
>511, c[0.076,96], d[0.001,0.000], g[0.457]
>512, c[0.060,99], d[0.001,0.000], g[0.459]
>513, c[0.042,97], d[0.001,0.000], g[0.488]
>514, c[0.053,99], d[0.001,0.000], g[0.501]
>515, c[0.030,99], d[0.001,0.000], g[0.518]
>516, c[0.229,94], d[0.001,0.000], g[0.522]
>517, c[0.127,96], d[0.001,0.000], g[0.504]
>518, c[0.051,97], d[0.001,0.000], g[0.482]
>519, c[0.071,95], d[0.001,0.000], g[0.466]
>520, c[0.036,98], d[0.001,0.000], g[0.470]
>521, c[0.025,99], d[0.001,0.000], g[0.443]
>522, c[0.011,100], d[0.0

In [ ]:
sgan = SGAN()
sgan.train ( epochs=1500, batch_size=100 , sample_interval=500,dataset= trainSet,testset = testSet)
#sgan.summarize_performance(499,testSet)

NameError: ignored